# Sensitivity Analysis

In [1]:
import os
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy.interpolate import griddata

plot_dir = "../plots/"

filename = "../simulation_data/robustness_check.csv"

df = pd.read_csv(filename)


# select columns
df_select = df[['total_steps','mean_purchase_position', 'Average Satisfaction', 'learning_rate_gen_ai', 'capacity_gen_ai','creativity_gen_ai', 'percentage_willing_to_share_info', 'num_customers', 'num_retailers', 'num_products','AIC Quartic (Sum)', 'AIC Quartic (Minimum)', 'Average Seller Rating']]

df_select.replace([-np.inf, np.inf], np.nan, inplace=True)

df_select.dropna(inplace=True)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/lx/qwfk5l855bq3cnpg8br590200000gn/T/ipykernel_2752/892682689.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select.replace([-np.inf, np.inf], np.nan, inplace=True)
/var/folders/lx/qwfk5l855bq3cnpg8br590200000gn/T/ipykernel_2752/892682689.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select.dropna(inplace=True)


In [2]:
# Remove rows where 'aic_quartic_original' is less than -450
df_select = df_select[df_select['AIC Quartic (Minimum)'] >= -450]

In [3]:
# df_select = df_select[df_select['num_customers'] <= 300]

# Group the data by 'learning_rate_gen_ai', 'capacity_gen_ai', and 'creativity_gen_ai'
grouped_df_test = df_select.groupby(['num_customers']).agg({
    'AIC Quartic (Sum)': 'mean',
    'AIC Quartic (Minimum)': 'mean',
    'Average Satisfaction': 'mean',
    'Average Seller Rating': 'mean',
}).reset_index()

# Print the head of the grouped DataFrame to check the results
grouped_df_test.head(10)


,num_customers,AIC Quartic (Sum),AIC Quartic (Minimum),Average Satisfaction,Average Seller Rating
0,10,-285.814179,-388.742555,1.551580,1.484600
1,25,-82.663607,-263.986873,2.822658,3.404780
2,50,-31.647295,-93.075107,2.045338,2.712913
3,100,-56.392389,-126.932233,1.966283,1.909041
4,200,-103.989224,-213.000703,2.167135,2.520396
5,500,-88.312339,-382.533006,1.257866,2.072897


In [4]:
# Group the data by 'learning_rate_gen_ai', 'capacity_gen_ai', and 'creativity_gen_ai'
grouped_df_test = df_select.groupby(['num_retailers']).agg({
    'AIC Quartic (Sum)': 'mean',
    'AIC Quartic (Minimum)': 'mean',
    'Average Satisfaction': 'mean',
    'Average Seller Rating': 'mean',
}).reset_index()

# Print the head of the grouped DataFrame to check the results
grouped_df_test.head(11)

,num_retailers,AIC Quartic (Sum),AIC Quartic (Minimum),Average Satisfaction,Average Seller Rating
0,10,-74.286232,-150.041716,2.199175,3.060468
1,20,-79.608643,-187.521429,1.859781,2.149312
2,40,-84.884571,-154.655711,2.275248,2.049554
3,80,-50.020233,-117.944841,2.103939,1.412759
4,100,-45.559825,-105.345113,2.179929,1.277042


In [5]:
# Group the data by 'learning_rate_gen_ai', 'capacity_gen_ai', and 'creativity_gen_ai'


grouped_df_test_2 = df_select.groupby(['num_products']).agg({
    'AIC Quartic (Sum)': 'mean',
    'AIC Quartic (Minimum)': 'mean',
    'Average Satisfaction': 'mean',
    'Average Seller Rating': 'mean',
}).reset_index()

# Print the head of the grouped DataFrame to check the results
grouped_df_test_2.head(50)

,num_products,AIC Quartic (Sum),AIC Quartic (Minimum),Average Satisfaction,Average Seller Rating
0,25,-40.627182,-119.857773,0.381398,0.562167
1,50,-23.895365,-138.665355,0.800937,0.867460
2,100,-78.501070,-175.386348,1.995639,2.165328
3,200,-51.765630,-130.834667,2.393291,2.515582
4,400,-61.172744,-127.277335,2.593983,1.756806


In [6]:
# rows_to_duplicate = grouped_df[grouped_df['percentage_willing_to_share_info'] == 0]
rows_to_duplicate = df_select[df_select['percentage_willing_to_share_info'] == 0]

duplicated_rows = pd.concat([rows_to_duplicate], ignore_index=True)

df_extended = pd.concat([df_select, duplicated_rows], ignore_index=True)


# Data Normalization

In [7]:
df_extended['AIC Quartic (Minimum) Original'] = df_extended['AIC Quartic (Minimum)']
columns_to_scale = ['Average Satisfaction', 'mean_purchase_position','percentage_willing_to_share_info', 'num_customers', 'num_retailers', 'num_products', 'AIC Quartic (Sum)', 'AIC Quartic (Minimum)', 'Average Seller Rating']
scaler = MinMaxScaler()
df_extended[columns_to_scale] = scaler.fit_transform(df_extended[columns_to_scale])
data_normalized_df = df_extended

In [8]:
# rename columns
data = data_normalized_df.rename(columns={
    'Average Satisfaction': 'avg_buyer_fitness',
    'Average Seller Rating': 'avg_seller_fitness',
    'percentage_willing_to_share_info': 'implement_gen_ai',
    'num_customers': 'n_customer',
    'num_retailers': 'n_seller',
    'num_products': 'n_product',
    'mean_purchase_position': 'mean_purchase_position',
    'AIC Quartic (Minimum)': 'aic_quartic_min',
    'AIC Quartic (Minimum) Original': 'aic_quartic_min_org',
    'AIC Quartic (Sum)': 'aic_quartic_sum'
})

data.head(10)

,total_steps,mean_purchase_position,avg_buyer_fitness,learning_rate_gen_ai,capacity_gen_ai,creativity_gen_ai,implement_gen_ai,n_customer,n_seller,n_product,aic_quartic_sum,aic_quartic_min,avg_seller_fitness,aic_quartic_min_org
0,100,0.326922,0.612403,0,0,0,0.0,0.030612,0.111111,0.2,0.746889,0.167669,0.871043,-377.600538
1,100,0.392512,0.786720,0,0,0,0.0,0.081633,0.111111,0.2,0.650983,0.616265,0.668861,-192.557573
2,100,0.420801,0.525084,0,0,0,0.0,0.081633,0.111111,0.2,0.641809,0.860567,0.652191,-91.784994
3,100,0.722349,0.043105,0,0,0,0.0,0.183673,0.111111,0.2,0.702484,0.926476,0.098459,-64.597816
4,100,0.778236,0.051128,0,0,0,0.0,0.183673,0.111111,0.2,0.748844,0.975678,0.215743,-44.302097
5,100,0.763284,0.013300,0,0,0,0.0,0.387755,0.111111,0.2,0.318662,0.432343,0.139217,-268.424515
6,100,0.622666,0.218943,0,0,0,0.0,0.000000,0.111111,0.2,0.731847,0.164872,0.038060,-378.754371
7,100,0.558773,0.220601,0,0,0,0.0,0.000000,0.111111,0.2,0.269004,0.253063,0.069423,-342.376243
8,100,0.216307,0.850416,0,0,0,0.0,0.030612,0.111111,0.2,0.666816,0.312442,0.940498,-317.882747
9,100,0.304300,0.629029,0,0,0,0.0,0.081633,0.111111,0.2,0.790230,0.947310,0.814682,-56.003913


In [9]:
# Selecting the independent variables/features for the model
# X = data[['gen_ai_learning_rate', 'gen_ai_creativity','gen_ai_capacity', 'implement_gen_ai', 'n_customer', 'n_seller', 'n_product']]
X = data[['n_customer', 'n_seller', 'n_product']]
# Adding a constant to the model (intercept)

X = sm.add_constant(X)
# Selecting the dependent variable/target
y = data['aic_quartic_min']

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# print model
print(model.summary())

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
                            OLS Regression Results                            
Dep. Variable:        aic_quartic_min   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     39.20
Date:                Mon, 06 May 2024   Prob (F-statistic):           3.44e-23
Time:                        12:38:31   Log-Likelihood:                -58.386
No. Observations:                 618   AIC:                             124.8
Df Residuals:                     614   BIC:                             142.5
Df Model:                           3                                         
Covariance Type:            nonrobust  